# Progetto MCS

Per la gestione della struttura dati e le operazioni elementari fra matrici è richiesto di partire da una libreria open-source, come Eigen, Armadillo, blas/lapack. Oppure, qualora il linguaggio di programmazione lo permetta utilizzare vettori e matrici già implementate al suo interno.


## Data Import

Impotazione delle librerie necessarie

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import mmread
from scipy.sparse import csr_matrix, tril
from scipy.sparse.linalg import spsolve_triangular

Recupero dei dati

In [ ]:
data = {
    "spa1" : 0,
    "spa2" : 0,
    "vem1" : 0,
    "vem2" : 0}
for x in data:
    data[x] = {
        "A" : mmread("data/" + x + ".mtx").tocsr(),
        "x" : 0,
        "b" : 0,
    }
    data[x]["x"] = np.array([1.0]*data[x]["A"].get_shape()[0])
    data[x]["b"] = np.array(data[x]["A"].dot(data[x]["x"]))
resTot = {}

Tolleranze e metodi

In [ ]:
## Tolleranze
tols = [10**(-4), 10**(-6), 10**(-8), 10**(-10)]

## Metodi
metods = ["Jacoby", "Gauss-Seidel", "Gradiente", "Gradiente Coniugato"]

Dizionario contente le classi risolutrici

In [ ]:
import jacoby_mcs as ja
import gauss_seidel as gs
import gradiente as gr
import gradiente_coniugato as grc

solver = {}
solver["Jacoby"] = ja
solver["Gauss-Seidel"] = gs
solver["Gradiente"] = gr
solver["Gradiente Coniugato"] = grc

Condizione di arresto: $\frac{||Ax^{(k)}-b||}{||b||}<tol$

Reminder: 

1.   **Iniziare le iterazioni con il vettore x nullo**
2.   **tol = [$10^{-4}, 10^{-6}, 10^{-8}, 10^{-10}]$**
3.   **Dichiarare di non essere giunti a convergenza se k > maxiter dove maxiter lo scegliamo (>= 20000)**




### Grafici e informazioni sulle matrici

Impostazione dei grafici

In [ ]:
import matplotlib.pylab as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 18]
plt.rcParams['figure.dpi'] = 100

Grafici delle matrici

In [ ]:
i = 221
for mt in data:
    plt.subplot(i)
    plt.spy(data[mt]["A"], markersize = 200/data[mt]["A"].shape[0])
    plt.title("Matrice " + mt, fontsize = 18 )
    i += 1
plt.suptitle("Matrici usate per il progetto", fontsize = 40)

info sulle matrici

In [ ]:
for mt in data:
    den = str(round((100 * data[mt]["A"].nnz) / (data[mt]["A"].shape[0]*data[mt]["A"].shape[1]), 2))
    print("La matrice " + mt + " è una matrice " + str(data[mt]["A"].shape[0]) + "x" + str(data[mt]["A"].shape[1]) + ", contiene " + str(data[mt]["A"].nnz) + " elementi diversi da 0 ed è quindi densa del " + den + "%")

### Preparazione grafico del residuo/errore relativo

In [ ]:
#PRIMA DI ESEGUIRLO CHIAMARE LE FUNZIONI IN FONDO
charts_results = {}
metods = {
    'jacobi': jacobi,
    'gauss_seidel': gauss_seidel,
    'gradiente': gradiente,
    'gradiente_coniugato': gradiente_coniugato
}
for metod in metods:    
    charts_results[metod] = {}
    for el in data:
        charts_results[metod][el] = []
        for tol in tols:
            chart_result = metods[metod](data[el]["A"], data[el]["b"], data[el]["x"], tol)
            charts_results[metod][el].append(chart_result)
charts_results



### GRAFICI SU RESIDUI PER NUMERO ITER

In [ ]:
strtols = ["1e-04", "1e-06", "1e-08", "1e-10"]
for metod in metods:
    for el in data:
        plt.plot(charts_results[metod][el][3]["residual_chart"])
        for index, chart in enumerate(charts_results[metod][el]):
            x=len(chart["residual_chart"])
            plt.plot(x, charts_results[metod][el][3]["residual_chart"][x-1], color='red', marker='o')
            plt.text(x, charts_results[metod][el][3]["residual_chart"][x-1], strtols[index], color='black', verticalalignment='bottom')

        # Etichette degli array
        plt.xlabel('ITERAZIONI')
        plt.ylabel('VALORE CRITERIO DI ARRESTO')
        plt.title('VALORI CRITERIO DI ARRESTO PER '+ metod.upper() + ' SULLA MATRICE ' + el.upper() )
        # Mostra il grafico
        plt.rcParams['figure.figsize'] = [15, 12]
        plt.savefig('VALORI CRITERIO DI ARRESTO PER '+ metod.upper() + ' SULLA MATRICE ' + el.upper())
        plt.show()

### GRAFICI SU ERRORI RELATIVI PER NUMERO ITER

In [ ]:
#max_length = max(len(chart["errrel_chart"]) for chart in charts_results['jacobi']['spa1'])
strtols = ["1e-04", "1e-06", "1e-08", "1e-10"]
for metod in metods:
    for el in data:
        plt.plot(charts_results[metod][el][3]["errrel_chart"])
        for index, chart in enumerate(charts_results[metod][el]):
            x=len(chart["errrel_chart"])
            plt.plot(x, charts_results[metod][el][3]["errrel_chart"][x-1], color='red', marker='o')
            plt.text(x, charts_results[metod][el][3]["errrel_chart"][x-1], strtols[index], color='black', verticalalignment='bottom')

        # Etichette degli array
        plt.xlabel('ITERAZIONI')
        plt.ylabel('ERRORE RELATIVO')
        plt.title('ERRORE RELATIVO PER '+ metod.upper() + ' SULLA MATRICE ' + el.upper() )
        plt.rcParams['figure.figsize'] = [15, 12]
        plt.savefig('ERRORE RELATIVO PER '+ metod.upper() + ' SULLA MATRICE ' + el.upper())
        # Mostra il grafico
        plt.show()


## Metodo di Jacobi

In [ ]:
import jacoby_mcs as ja
method = "Jacoby"
resTot = {}
resTot[method] = {}
for el in data:
    resTot[method][el] = []
    for tol in tols:
        res = ja.solve(data[el]["A"], data[el]["b"], data[el]["x"], tol)
        resTot[method][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })

resTot

### Grafici

In [ ]:
tmpDF = {}
iterDF = {}
errDF = {}
br = {}
for tol in tols:
    br[str(tol)] = []
    tmpDF[str(tol)] = []
    iterDF[str(tol)] = []
    errDF[str(tol)] = []
for mt in resTot[method]:
    for el in resTot[method][mt]:
        tmpDF[str(el["tol"])].append(el["time"])
        iterDF[str(el["tol"])].append(el["nIter"])
        errDF[str(el["tol"])].append(el["eRel"])
    
barWidth = 1/(len(tmpDF)+1)

ks = list(br.keys())
br[ks[0]] = np.arange(len(tols))
for i in range(1, len(br)):
    br[ks[i]] =  br2 = [round(x + barWidth, 2) for x in br[ks[i-1]]]

colors = ["r", "g", "b", "y"]

mts = list(data.keys())

plt.rcParams['figure.figsize'] = [9, 6]


Istogrammi tempi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], tmpDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi iterazioni

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], iterDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi errori relativi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], errDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

## Metodo di Gauß-Seidel

1. $r^{(k)} = b − Ax^{(k)}$
2. Sostituzione in avanti Py = r^{(k)}
3. $x^{(k+1)} = y^{(k)} + y$


In [ ]:
import gauss_seidel as gs
method = "Gauss-Seidel"
resTot[method] = {}
for el in data:
    resTot[method][el] = []
    for tol in tols:
        res = gs.solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, vectX=data[el]["x"])
        resTot[method][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })

resTot

### Grafici

In [ ]:
tmpDF = {}
iterDF = {}
errDF = {}
br = {}
for tol in tols:
    br[str(tol)] = []
    tmpDF[str(tol)] = []
    iterDF[str(tol)] = []
    errDF[str(tol)] = []
for mt in resTot[method]:
    for el in resTot[method][mt]:
        tmpDF[str(el["tol"])].append(el["time"])
        iterDF[str(el["tol"])].append(el["nIter"])
        errDF[str(el["tol"])].append(el["eRel"])
    
barWidth = 1/(len(tmpDF)+1)

ks = list(br.keys())
br[ks[0]] = np.arange(len(tols))
for i in range(1, len(br)):
    br[ks[i]] =  br2 = [round(x + barWidth, 2) for x in br[ks[i-1]]]

colors = ["r", "g", "b", "y"]

mts = list(data.keys())

plt.rcParams['figure.figsize'] = [9, 6]


Istogrammi tempi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], tmpDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi iterazioni

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], iterDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi errori relativi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], errDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

## Metodo del Gradiente


1. $r^{(k)} = b -Ax^{(k)}$
2. $y^{(k)} = Ar^{(k)}$
3. $a = (r^{(k)})^tr^{(k)}$
4. $b = (r^{(k)})^ty^{(k)}$
5. $\alpha_k = a/b$
6. $x^{(k+1)} = r^{(k)} \alpha_kr^{(k)}$

In [ ]:
import gradiente as gr

method = "Gradiente"
solver = {}
solver[method] = gr
resTot[method] = {}
for el in data:
    resTot[method][el] = []
    for tol in tols:
        res = solver[method].solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, vectX=data[el]["x"])
        resTot[method][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })
resTot

### Grafici

In [ ]:
tmpDF = {}
iterDF = {}
errDF = {}
br = {}
for tol in tols:
    br[str(tol)] = []
    tmpDF[str(tol)] = []
    iterDF[str(tol)] = []
    errDF[str(tol)] = []
for mt in resTot[method]:
    for el in resTot[method][mt]:
        tmpDF[str(el["tol"])].append(el["time"])
        iterDF[str(el["tol"])].append(el["nIter"])
        errDF[str(el["tol"])].append(el["eRel"])
    
barWidth = 1/(len(tmpDF)+1)

ks = list(br.keys())
br[ks[0]] = np.arange(len(tols))
for i in range(1, len(br)):
    br[ks[i]] =  br2 = [round(x + barWidth, 2) for x in br[ks[i-1]]]

colors = ["r", "g", "b", "y"]

mts = list(data.keys())

plt.rcParams['figure.figsize'] = [9, 6]


Istogrammi tempi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], tmpDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi iterazioni

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], iterDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi errori relativi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], errDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

## Metodo del Gradiente coniugato

- Un vettore ottimale rispetto a una direzione d se d*r(k)=0
- x(k+1) è ottimale rispetto a r(k+1)
- x(k+1) = x(k) + a(k)d(k)
- a(k) = ( d(k)^t * r(k) ) / ( d(k)^t * Ad(k) )
- d(k+1) = r(k+1) - b(k)*d(k)
- b(k) = ( d(k)^t * Ar(k+1) ) / ( d(k)^t * Ad(k) )



In [ ]:
import gradiente_coniugato as grc
method = "Gradiente Coniugato"
resTot = {}
resTot[method] = {}
for el in data:
    resTot[method][el] = []
    for tol in tols:
        res = grc.solve(data[el]["A"], data[el]["b"], data[el]["x"], tol)
        resTot[method][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })

resTot

### Grafici

In [ ]:
tmpDF = {}
iterDF = {}
errDF = {}
br = {}
for tol in tols:
    br[str(tol)] = []
    tmpDF[str(tol)] = []
    iterDF[str(tol)] = []
    errDF[str(tol)] = []
for mt in resTot[method]:
    for el in resTot[method][mt]:
        tmpDF[str(el["tol"])].append(el["time"])
        iterDF[str(el["tol"])].append(el["nIter"])
        errDF[str(el["tol"])].append(el["eRel"])
    
barWidth = 1/(len(tmpDF)+1)

ks = list(br.keys())
br[ks[0]] = np.arange(len(tols))
for i in range(1, len(br)):
    br[ks[i]] =  br2 = [round(x + barWidth, 2) for x in br[ks[i-1]]]

colors = ["r", "g", "b", "y"]

mts = list(data.keys())

plt.rcParams['figure.figsize'] = [9, 6]


Istogrammi tempi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], tmpDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], tmpDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('tempistiche', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi iterazioni

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], iterDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], iterDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('iterazioni', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

Istogrammi errori relativi

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])], errDF[str(tols[i])], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(tols))],
        mts)
plt.title(method)

plt.legend()
plt.show()

In [ ]:
for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][:2], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[:2]))],
        mts[:2])
plt.title(method + " ~ SPA")

plt.legend()
plt.show()

for i in range(len(tols)):
    plt.bar(br[str(tols[i])][:2], errDF[str(tols[i])][2:], 
            color = colors[i], width = barWidth,
                edgecolor ='grey', label =str(tols[i]))

plt.xlabel('Matrici', fontweight ='bold', fontsize = 15)
plt.ylabel('errori relativi', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(mts[2:]))],
        mts[2:])
plt.title(method + " ~ VEM")

plt.legend()
plt.show()

## Risultati totali

In [ ]:
resTot = {}
for mth in solver:    
    resTot[mth] = {}
    for el in data:
        resTot[mth][el] = []
        for tol in tols:
            res = solver[mth].solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, vectX=data[el]["x"])
            resTot[mth][el].append({
                "tol" : tol,
                "nIter" : res["nIter"],
                "time" : res["time"],
                "eRel" : res["eRel"] 
            })
resTot

## FUNZIONI DEI FILE COMODE QUA PER IL NOTEBOOK

In [ ]:
def relative_error(x, xk1):
    return np.linalg.norm(np.subtract(xk1, x))/np.linalg.norm(x)

In [ ]:
def gradiente(mtxA, vectB, vectX, tol):
    charts= {
        "residual_chart": [],
        "errrel_chart": [],
    }
    # Variabili
    k = 0
    vectX1 = np.zeros(mtxA.shape[0])
    residual = vectB - mtxA.dot(vectX1)

    # Funzione
    while np.linalg.norm(residual)/np.linalg.norm(vectB) >= tol and k <= 20000:
        k += 1
        y = mtxA.dot(residual)
        a = residual.T.dot(residual)
        b = residual.T.dot(y)
        alpha = a/b

        vectX1 = vectX1 + alpha * residual
        charts["residual_chart"].append(np.linalg.norm(residual)/np.linalg.norm(vectB))
        charts["errrel_chart"].append(relative_error(vectX, vectX1))
        residual = vectB - mtxA.dot(vectX1)
    if k > 20000:
        if np.linalg.norm(residual)/np.linalg.norm(vectB) >= tol:
            print("superato il numero massimo di iterazioni")
    # Risultato
    return charts

In [ ]:
def gauss_seidel(mtxA, vectB, vectX, tol):
    # Variabili
    charts= {
        "residual_chart": [],
        "errrel_chart": [],
    }
    maxIter = 20000
    mtxP = tril(mtxA, format="csr")
    k = 0
    vectX1 = np.zeros(mtxA.shape[0])
    residual = vectB - mtxA.dot(vectX1)

    # Funzione
    while np.linalg.norm(residual)/np.linalg.norm(vectB) >= tol and k <= maxIter:
        k += 1
        vectX1 = vectX1 + spsolve_triangular(mtxP, residual, lower=True)
        charts["residual_chart"].append(np.linalg.norm(residual)/np.linalg.norm(vectB))
        charts["errrel_chart"].append(relative_error(vectX, vectX1))
        residual = vectB - mtxA.dot(vectX1)

    if k > 20000:
        if np.linalg.norm(residual)/np.linalg.norm(vectB) > tol:
            print("superato il numero massimo di iterazioni")
    # Risultato
    return charts

In [ ]:
def gradiente_coniugato(A, b, x, tol):
    charts= {
        "residual_chart": [],
        "errrel_chart": [],
    }
    niter = 0
    new_vector = np.asarray([0]*len(x))
    residual = b - A.dot(new_vector)
    dir = residual.copy()
    while np.linalg.norm(residual)/np.linalg.norm(b) >= tol and niter <= 20000:
        y = A.dot(dir)
        z = A.dot(residual)
        ak = (np.dot(dir, residual)) / (np.dot(dir, y))
        new_vector = new_vector + (ak * dir)
        residual = b - A.dot(new_vector)
        w = A.dot(residual)
        bk = (np.dot(dir, w)) / (np.dot(dir, y))
        dir = residual - (bk*dir)
        charts["residual_chart"].append(np.linalg.norm(residual)/np.linalg.norm(b))
        charts["errrel_chart"].append(relative_error(x, new_vector))
        residual = b - A.dot(new_vector)
        niter = niter + 1

    if niter > 20000:
        if np.linalg.norm(residual)/np.linalg.norm(b) >= tol:
            print("superato il numero massimo di iterazioni")
    # Risultato
    return charts


In [ ]:
def jacobi(A, b, x, tol):
    charts= {
        "residual_chart": [],
        "errrel_chart": [],
    }
    niter = 0
    new_vector = np.asarray([0]*len(x))
    inverted_p_matrix = 1/A.diagonal()
    residual = b - A.dot(new_vector)
    while np.linalg.norm(residual)/np.linalg.norm(b) >= tol and niter <= 20000:
        new_vector = new_vector + (inverted_p_matrix * (residual))
        charts["residual_chart"].append(np.linalg.norm(residual)/np.linalg.norm(b))
        charts["errrel_chart"].append(relative_error(x, new_vector))
        residual = b - A.dot(new_vector)
        niter = niter + 1
#    return {"iter": niter, "err_rel": relative_error(x, new_vector)}
    if niter > 20000:
        if np.linalg.norm(residual)/np.linalg.norm(b) >= tol:
            print("superato il numero massimo di iterazioni")
    # Risultato
    return charts